# NER with BERT in Spark NLP

## Installation

In [19]:
# This is only to setup PySpark and Spark NLP on Colab
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2023-10-30 15:56:55--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2023-10-30 15:56:55--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8000::154, 2606:50c0:8001::154, 2606:50c0:8002::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1191 (1,2K) [text/plain]
Saving to: ‘STDOUT’

-                   100%[===================>]   1,16K  --.-KB/s    in 0s      

2023-10-30 15:56:56 (56,8 MB/s) - written to stdout [1191/1

In [4]:
# !pip install sparknlp

## Import libraries and download datasets

In [20]:
import sparknlp
from sparknlp.annotator import *
from sparknlp.base import *

In [6]:
# if you have GPU
# spark = sparknlp.start(gpu=True)
spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.

/Users/ciprian.turcu/Library/Caches/pypoetry/virtualenvs/ml-engineer-assignment1-qOUDlTmM-py3.11/lib/python3.11/site-packages/pyspark/bin/spark-class: line 96: CMD: bad array subscript
head: illegal line count -- -1


RuntimeError: Java gateway process exited before sending its port number

In [7]:
from urllib.request import urlretrieve

urlretrieve('https://github.com/JohnSnowLabs/spark-nlp/raw/master/src/test/resources/conll2003/eng.train',
           'eng.train')

urlretrieve('https://github.com/JohnSnowLabs/spark-nlp/raw/master/src/test/resources/conll2003/eng.testa',
           'eng.testa')


URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)>

In [8]:
with open("eng.train") as f:
    c=f.read()

print (c[:200])

FileNotFoundError: [Errno 2] No such file or directory: 'eng.train'

## Building NER pipeline

In [9]:
from sparknlp.training import CoNLL

training_data = CoNLL().readDataset(spark, './eng.train')
training_data.show(3)

AttributeError: Cannot load _jvm from SparkContext. Is SparkContext initialized?

In [10]:
training_data.count()

NameError: name 'training_data' is not defined

### Loading Bert

In Spark NLP, we have four pre-trained variants of BERT: bert_base_uncased , bert_base_cased , bert_large_uncased , bert_large_cased, and many Smaller BERT models available on our [Models Hub](https://nlp.johnsnowlabs.com/models?q=bert&task=Embeddings). Which one to use depends on your use case, train set, and the complexity of the task you are trying to model.

In the code snippet above, we basically load the bert_base_cased version from Spark NLP public resources and point the sentence and token columns in   setInputCols(). In short, BertEmbeddings() annotator will take sentence and token columns and populate Bert embeddings in bert column. In general, each word is translated to a 768-dimensional vector.

As explained by the authors of official BERT paper, different BERT layers capture different information. The last layer is too closed to the target functions (i.e. masked language model and next sentence prediction) during pre-training, therefore it may be biased to those targets. 

In [11]:
# we use BERT Tiny
bert_annotator = BertEmbeddings.pretrained('small_bert_L2_128', 'en') \
 .setInputCols(["sentence",'token'])\
 .setOutputCol("bert")\
.setBatchSize(8)

small_bert_L2_128 download started this may take some time.


AttributeError: Cannot load _jvm from SparkContext. Is SparkContext initialized?

In [12]:
from sparknlp.training import CoNLL

test_data = CoNLL().readDataset(spark, './eng.testa')

test_data = bert_annotator.transform(test_data)

test_data.show(3)

AttributeError: Cannot load _jvm from SparkContext. Is SparkContext initialized?

In [13]:
# let's transform and save our test dataset for evaluation
test_data.write.parquet("test_withEmbeds.parquet")

NameError: name 'test_data' is not defined

In [14]:
test_data.select("bert.result","bert.embeddings",'label.result').show()

NameError: name 'test_data' is not defined

In [15]:
nerTagger = NerDLApproach()\
  .setInputCols(["sentence", "token", "bert"])\
  .setLabelColumn("label")\
  .setOutputCol("ner")\
  .setMaxEpochs(5)\
  .setLr(0.001)\
  .setPo(0.005)\
  .setBatchSize(32)\
  .setEvaluationLogExtended(True) \
  .setEnableOutputLogs(True)\
  .setTestDataset("test_withEmbeds.parquet")

pipeline = Pipeline(
    stages = [
    bert_annotator,
    nerTagger
  ])

AttributeError: Cannot load _jvm from SparkContext. Is SparkContext initialized?

You can also set learning rate ( setLr ), learning rate decay coefficient ( setPo ), setBatchSize and setDropout rate. Please see the [official APIs](https://nlp.johnsnowlabs.com/api/com/johnsnowlabs/nlp/annotators/ner/dl/NerDLApproach.html) for the entire list. 

In [16]:
%%time

ner_model = pipeline.fit(training_data)

NameError: name 'pipeline' is not defined

In [17]:
!ls -l /root/annotator_logs/

ls: /root/annotator_logs/: No such file or directory


In [18]:
!cat /root/annotator_logs/NerDLApproach_16a0e7b3577f.log

cat: /root/annotator_logs/NerDLApproach_16a0e7b3577f.log: No such file or directory


**Some notes:**
- we used the smallest BERT model called BERT Tiny
- it's very small and requires less memory among Transformers
- if you have more memory or access to accelerated hardware please choose a larger BERT model for higher accuracy
- you can also set higher Epoch to reach our STOA metrics

We chose the smallest BERT model with only 5 Epochs for the sake of this tutorial within this small Colab VM

In [ ]:
# let's save our trained NER model on disk
# so we can load it in a new session or move it to another location
# since we fit NerDL model inside the pipeline, we can access it via stages
ner_model.stages[1].write().overwrite().save('./NER_bert_20200219')

In [ ]:
test_data.show()

In [ ]:
# let's only feed sentence and token from our test dataset
predictions = ner_model.transform(test_data.select("sentence", "token", "label"))
predictions.show(3)

In [ ]:
predictions.select('token.result','label.result','ner.result').show(truncate=40)

In [ ]:
predictions.printSchema()

In [ ]:
import pyspark.sql.functions as F

predictions.select(F.explode(F.arrays_zip('token.result','label.result','ner.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("token"),
        F.expr("cols['1']").alias("ground_truth"),
        F.expr("cols['2']").alias("prediction")).show(truncate=False)

# Convert to Pandas

In [ ]:
import pandas as pd

df = predictions.select('token.result','label.result','ner.result').toPandas()

df

## Prediction Pipeline

In [ ]:
from pyspark.ml import Pipeline

document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentence = SentenceDetector()\
    .setInputCols(['document'])\
    .setOutputCol('sentence')

token = Tokenizer()\
    .setInputCols(['sentence'])\
    .setOutputCol('token')

bert = BertEmbeddings.pretrained('small_bert_L2_128', 'en') \
 .setInputCols(["sentence",'token'])\
 .setOutputCol("bert")\
 .setCaseSensitive(True)

loaded_ner_model = NerDLModel.load("NER_bert_20200219")\
 .setInputCols(["sentence", "token", "bert"])\
 .setOutputCol("ner")

converter = NerConverter()\
  .setInputCols(["document", "token", "ner"])\
  .setOutputCol("ner_span")

ner_prediction_pipeline = Pipeline(
    stages = [
        document,
        sentence,
        token,
        bert,
        loaded_ner_model,
        converter])

In [ ]:
empty_data = spark.createDataFrame([['']]).toDF("text")

empty_data.show()

In [ ]:
prediction_model = ner_prediction_pipeline.fit(empty_data)


In [ ]:
text = "Peter Parker is a nice guy and lives in New York."
sample_data = spark.createDataFrame([[text]]).toDF("text")
sample_data.show()

In [ ]:

preds = prediction_model.transform(sample_data)

preds.show()

In [ ]:
preds.select('ner_span.result').take(1)

In [ ]:

preds.select(F.explode(F.arrays_zip("ner_span.result","ner_span.metadata")).alias("entities")) \
.select(F.expr("entities['0']").alias("chunk"),
        F.expr("entities['1'].entity").alias("entity")).show(truncate=False)

In [ ]:
document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentence = SentenceDetector()\
    .setInputCols(['document'])\
    .setOutputCol('sentence')

token = Tokenizer()\
    .setInputCols(['sentence'])\
    .setOutputCol('token')

loaded_ner_model = NerDLModel.load("NER_bert_20200219")\
 .setInputCols(["sentence", "token", "glove"])\
 .setOutputCol("ner")

converter = NerConverter()\
  .setInputCols(["document", "token", "ner"])\
  .setOutputCol("ner_span")

glove_ner_prediction_pipeline = Pipeline(
    stages = [
        document,
        sentence,
        token,
        glove,
        loaded_ner_model,
        converter])

In [ ]:
glove_prediction_model = glove_ner_prediction_pipeline.fit(empty_data)

In [ ]:

preds = glove_prediction_model.transform(sample_data)

preds.show()

In [ ]:

preds.select(F.explode(F.arrays_zip("ner_span.result","ner_span.metadata")).alias("entities")) \
.select(F.expr("entities['0']").alias("chunk"),
        F.expr("entities['1'].entity").alias("entity")).show(truncate=False)

### Pretrained Pipelines

In [ ]:
from sparknlp.pretrained import PretrainedPipeline

pretrained_pipeline = PretrainedPipeline('recognize_entities_dl', lang='en')

#onto_recognize_entities_sm
#explain_document_dl

In [ ]:
text = "The Mona Lisa is a 16th century oil painting created by Leonardo. It's held at the Louvre in Paris."

result = pretrained_pipeline.annotate(text)

list(zip(result['token'], result['ner']))

In [ ]:
pretrained_pipeline2 = PretrainedPipeline('explain_document_dl', lang='en')


In [ ]:
text = "The Mona Lisa is a 16th centry oil painting created by Leonrdo. It's held at the Louvre in Paris."

result2 = pretrained_pipeline2.annotate(text)

result2
list(zip(result2['token'],  result2['checked'], result2['pos'], result2['ner'],  result2['lemma'],  result2['stem']))

In [ ]:
xx= pretrained_pipeline2.fullAnnotate(text)

[(n.result, n.metadata['entity']) for n in xx['ner_span']]

### with Glove Embeddings

In [ ]:
glove = WordEmbeddingsModel().pretrained() \
 .setInputCols(["sentence",'token'])\
 .setOutputCol("glove")\
 .setCaseSensitive(False)

test_data = CoNLL().readDataset(spark, './eng.testa')

test_data = glove.transform(test_data.limit(1000))

test_data.write.parquet("test_withGloveEmbeds.parquet")


In [ ]:
nerTagger.setInputCols(["sentence", "token", "glove"])
nerTagger.setTestDataset("test_withGloveEmbeds.parquet")

glove_pipeline = Pipeline(
    stages = [
    glove,
    nerTagger
  ])

In [ ]:
%%time

ner_model_v3 = glove_pipeline.fit(training_data.limit(1000))

In [ ]:
predictions_v3 = ner_model_v3.transform(test_data.limit(10))

# test_data.sample(False,0.1,0)

predictions_v3.select(F.explode(F.arrays_zip('token.result','label.result','ner.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("token"),
        F.expr("cols['1']").alias("ground_truth"),
        F.expr("cols['2']").alias("prediction")).show(truncate=False)

In [ ]:
np.array (predictions.select('token.result').take(1))[0][0]

In [ ]:
import pandas as pd

tokens = np.array (predictions.select('token.result').take(1))[0][0]
ground = np.array (predictions.select('label.result').take(1))[0][0]
label_bert_0 = np.array (predictions.select('ner.result').take(1))[0][0]
label_bert_2 = np.array (predictions_v2.select('ner.result').take(1))[0][0]
label_glove = np.array (predictions_v3.select('ner.result').take(1))[0][0]

pd.DataFrame({'token':tokens,
              'ground':ground,
              'label_bert_0':label_bert_0,
              'label_bert_2':label_bert_2,
              'label_glove':label_glove})

## Using your own custom Word Embedding

In [ ]:
custom_embeddings = WordEmbeddings()\
  .setInputCols(["sentence", "token"])\
  .setOutputCol("glove")\
  .setStoragePath('/Users/vkocaman/cache_pretrained/PubMed-shuffle-win-2.bin', "BINARY")\
.setDimension(200)

In [ ]:
custom_embeddings.fit(training_data.limit(10)).transform(training_data.limit(10)).show()

## creating your own CoNLL dataset

In [ ]:
import json
import os
from pyspark.ml import Pipeline
from sparknlp.base import *
from sparknlp.annotator import *
import sparknlp

spark = sparknlp.start()

def get_ann_pipeline ():
    
    document_assembler = DocumentAssembler() \
        .setInputCol("text")\
        .setOutputCol('document')

    sentence = SentenceDetector()\
        .setInputCols(['document'])\
        .setOutputCol('sentence')\
        .setCustomBounds(['\n'])

    tokenizer = Tokenizer() \
        .setInputCols(["sentence"]) \
        .setOutputCol("token")

    pos = PerceptronModel.pretrained() \
              .setInputCols(["sentence", "token"]) \
              .setOutputCol("pos")
    
    embeddings = WordEmbeddingsModel.pretrained()\
          .setInputCols(["sentence", "token"])\
          .setOutputCol("embeddings")

    ner_model = NerDLModel.pretrained() \
          .setInputCols(["sentence", "token", "embeddings"]) \
          .setOutputCol("ner")

    ner_converter = NerConverter()\
      .setInputCols(["sentence", "token", "ner"])\
      .setOutputCol("ner_chunk")

    ner_pipeline = Pipeline(
        stages = [
            document_assembler,
            sentence,
            tokenizer,
            pos,
            embeddings,
            ner_model,
            ner_converter
        ]
    )

    empty_data = spark.createDataFrame([[""]]).toDF("text")

    ner_pipelineFit = ner_pipeline.fit(empty_data)

    ner_lp_pipeline = LightPipeline(ner_pipelineFit)

    print ("Spark NLP NER lightpipeline is created")

    return ner_lp_pipeline


In [ ]:
conll_pipeline = get_ann_pipeline ()

In [ ]:
parsed = conll_pipeline.annotate ("Peter Parker is a nice guy and lives in New York.")
parsed

In [ ]:
conll_lines=''

for token, pos, ner in zip(parsed['token'],parsed['pos'],parsed['ner']):

    conll_lines += "{} {} {} {}\n".format(token, pos, pos, ner)


print(conll_lines)